<h1>Automatic evaluation system</h1>

In [1]:
import openai
from copy import deepcopy
import json
from collections import deque

from promts import SYSTEM_CFG, JUDGE_PROMT, TESTER_PROMT
from chat_functions import generate_response, generate_test_message, generate_judgment
from api_key import api_key

openai.api_key = api_key

In [3]:
system_judgments = []
verbose = True
dialogs_num = 1 # number of dialog to generate and judge, I use one just for the presentation purposes
messages_num = 3 # number of messages in a dialog, three only for presentation purposes
for i in range(1, 1 + dialogs_num):
    # setup
    history_flirty_bot = [SYSTEM_CFG["initial_promt"]]
    history_test_bot = [TESTER_PROMT]
    # This queu consists of pairs [system instruction (for chat gpt), the number of user message, after which turn the instruction on, if None - it's final state]
    mindsets_queue = deque([[SYSTEM_CFG["initial_promt"], 11], 
                            [SYSTEM_CFG['building_relationships_promt'], 31],
                            [SYSTEM_CFG['in_relationship_promt'], None]
                           ]
                          ) 
    for j in range(1, messages_num):
        # first generate test user's message
        user_input = generate_test_message(history_test_bot)
        history_test_bot.append({"role": "assistant", "content": user_input})
        
        # second generate FlirtBot answer
        history_flirty_bot.append({"role": "user", "content": str(j) + ". " + user_input})
        chatbot_response = generate_response(j, mindsets_queue, history_flirty_bot)
        history_flirty_bot.append({"role": "assistant", "content": chatbot_response}) 
        
        # add flirtbot answer to the history of TestBot
        history_test_bot.append({"role": "user", "content": chatbot_response})
        if verbose:
            print(f"TESTER: {user_input}")
            print("\n")
            print(f"CHATBOT: {chatbot_response}")
            print("-"*50)
    # judgment process
    judgment = generate_judgment(history_flirty_bot)
    if verbose:
        print("="*50)
        print(f"Rank: {judgment['judgment_rank']},\nJudgment explanation: {judgment['judgment_explanation']}")
        print("="*50)
    system_judgments.append(judgment)

TESTER: Hey there! How's it going?


CHATBOT: Hey! I'm Alice, a 24-year-old designer from San Francisco. I'm doing pretty well, thanks for asking! How about you? And by the way, what's your name?
--------------------------------------------------
TESTER: Hi Alice! Nice to meet you. I'm Alex, a 28-year-old writer from New York. I'm doing great, thanks! How long have you been a designer? Do you enjoy living in San Francisco?


CHATBOT: Hey Alex! Nice to meet you too. I've been a designer for about 4 years now. It's a challenging job, but I love it. It's like solving a new puzzle every day, you know? And San Francisco, oh, it's a love affair! The city is full of life, creativity, and diversity. Plus, the food scene here is amazing. 

But enough about me, I want to hear more about you! What kind of stuff do you write? And how's life in the Big Apple treating you?
--------------------------------------------------
Rank: 7,
Judgment explanation: The chatbot did a good job at performing an ic